#Import necessari

In [ ]:
import numpy as np
import re
from tabulate import tabulate

#English Context-free grammar

In [ ]:
English_Grammar = {
      # Grammatica che produce le frasi seguenti
    "S": ["NP VP", "ANP VP", "book", "include", "prefer", "Verb NP", "VNP PP", "Verb PP", "VP PP"],
    "ANP": ["Aux NP"],
    "NP": ["I", "She", "Me", "TWA", "Houston", "Det Nominal"], 
    "Nominal": ["book", "flight", "meal", "money", "morning", "Nominal Noun", "Nominal PP"], 
    "Noun": ["book", "flight", "morning"],
    "Det": ["that", "this", "the", "a"],
    "VP": ["Verb NP", "VNP PP", "Verb PP", "VP PP"],
    "Verb": ["book", "include", "prefer"],
    "Aux": ["does"], 
    "VNP": ["Verb NP"],
    "PP": ["Prep NP"],
    "Prep": ["from", "to", "on", "near", "through"]
}
"""
Le seguenti frasi sono:
  - Book the flight through Houston
  - Does she prefer a morning flight
"""
English_phrases = ["Book the flight through Houston", "Does she prefer a morning flight"]


#Dothraki Context-free grammar

In [ ]:
Dothraki_Grammar = {
    # Grammatica che produce le frasi seguenti
    "S": ["AV PN", "NP VP", "NP Noun", "NP Verb"],
    "AV": ["ANP Verb" ],
    "ANP":["Aux NP2"],
    "Aux":["hash"], 
    "NP": [ "yera", "anha"], # nominativo
    "NP2": ["yer"], # accusativo
    "PN": ["Prep Noun"],
    "VP": ["Verb NP"],
    "Prep": ["ki"], 
    "Noun": ["Dothraki", "gavork"],
    "Verb": ["astoe", "zhilak"]
}
"""
Le seguenti frasi sono:
  - Do you speak Dothraki ?
  - I love you
  - I'm hungry
"""
Dothraki_phrases = ["Hash yer astoe ki Dothraki?", "Anha zhilak yera", "Anha gavork"]

#Funzioni ausiliarie CKY

In [ ]:
def find_matches(CFG, list1, list2, grammar_rules):
    """
    Trova tutte le possibili combinazioni di regole a partire dalle 2 liste in input
    Restituisce:
    - una matrice di tutte le terne dei match trovati tra le varie combinazioni (es: [[VP, 4, 5], [PP, 6, 5]])
    """
    ret_matches =  np.empty((0,3), int)
    
    for idx1 in list1: 
        for idx2 in list2: 
            A = get_rules(CFG, idx1, idx2, grammar_rules)
            ret_matches = np.append(ret_matches, A,0)

    return ret_matches

In [ ]:
def save_rules(rules, word, matrix, row, col, index, grammar_rules):
    """
    Salva le regole trovate, in particolare:
        - salva la lista di chiavi create, nella cella corrente della matrice
        - per ognuna, crea una nuova entry nel dizionario
        - word: se è None allora sto ragionando su produzioni A->B,C, in alternativa sto 
          valutando un terminale ad esempio "flight"
        - index: a che chiave del dizionario grammar_rules sto scrivendo le nuove regole
    """
    #per ogni regola
    """
      Nel primo caso ho una produzione Det=that nel secondo caso invece ho produzioni 
      piu complesse come A->BC quindi salvo semplicemente le regole in grammar_rules
        
    """


    for rule in rules:
        #se la cella della matrice che conterrà la matrice finale (matrix) è vuota
        if not matrix[row,col]:
            #inserisco un array che continee solo l'elemento index
            matrix[row,col] = [index]
        else:
            #Se invece è piena aggiungo index alla lista in essa contenuta
            list_id = matrix[row,col]
            list_id.append(index)
            matrix[row,col] = list_id
        grammar_rules[index] = [rule[0], word, None] if word is not None else [rule[0], rule[1], rule[2]]
        index += 1
    
    return index

grammar_rules {0: ['S', 'Book', None], 1: ['Nominal', 'Book', None], 2: ['Noun', 'Book', None], 3: ['Verb', 'Book', None], 4: ['Det', 'the', None], 5: ['Nominal', 'flight', None], 6: ['Noun', 'flight', None], 7: ['NP', '4', '5']}

In [ ]:
"""
  Restituisce le produzioni della grammatica:
  Input:
  -CFG -> la grammatica che stiamo usando
  -id_first -> la parola corrente 
  -id_second -> una seconda parola (None opzionale)
  -grammar_rules -> insieme di regole prodotte
  Output:
    - una lista vuota, se non trova match
    - una terna [left, id_P, id_P] o [[left, T, None]] (es: [S, Book, None])
    - una lista di terne (tipicamente per i T) (es: [[S, Book, None], [Noun, Book, None]])

"""
def get_rules(CFG, id_first, id_second, grammar_rules, is_terminal = False): 

    # np.empty crea un nuovo array di grandezza 0-righe e 3-colonne senza inizilizzare i valori 
    ret_rules =  np.empty((0,3), int)
    
    if is_terminal:
        word = id_first
    else:
        first = grammar_rules[id_first][0]
        second = grammar_rules[id_second][0]
        #se esiste second la concateno altrimenti lascio solo first
        word = first + " " + second if second else first
    #print("Is Terminal: " + str(is_terminal))    
    #print("Word: " + word)
    for key, value in CFG.items():
      # se la parola è nella parte destra della regola
      # in altre parole se word appartiene alla gramamtica, inizializzo l'array ret_rules
        for row in value: 
            if (word.lower() == row.lower()):
                ret_rules = np.append(ret_rules, np.array([[key, id_first, id_second]]),0)
    return ret_rules

#CKY

In [ ]:
def CKY(words, grammar):
    """
    Algoritmo CKY che prende in input:
    -la frase prodotta dalla grammatica English/Dothraki dopo aver eseguito lo split in parole
    -la grammatica
    """
    #chiave del dizionario grammar_rules a cui scrivo le nuove regole
    key = 0     
    # grammar_rules è un dizionario con tutte le produzioni
    """
    Esempio
    grammar_rules {0: ['S', 'Book', None], 1: ['Nominal', 'Book', None], 2: ['Noun', 'Book', None], 3: ['Verb', 'Book', None], 4: ['Det', 'the', None], 5: ['Nominal', 'flight', None], 6: ['Noun', 'flight', None], 7: ['NP', '4', '5']}
    """
    #insieme delle produzioni della grammatica in questione
    grammar_rules = {}
    # np.full ritorna un nuovo array di lunghezza e tipo uguali a quelli messi nel campo fill 
    #in questo caso facciamo un nuovo array di None
    parsing_matrix = np.full((len(words),len(words)+1), None)
    for j in range(1, len(words)+1):
       #assegno la parola in posizione 0 (words[j-1]) che sarà una delle parole della frase da esaminare
        current_word = words[j-1] 
        rules = get_rules(grammar, current_word, None, grammar_rules, True)
        key = save_rules(rules, current_word, parsing_matrix, j-1, j, key, grammar_rules)

        #range(start, stop, step) parto da j-2, arrivo a -1 e tolgo 1 ogni passo
        for i in range(j-2, -1, -1):
            for k in range(i+1, j):

                B = parsing_matrix[i,k]
                #print(B)
                C = parsing_matrix[k,j]
                #print("C: "+str(C))
                if (B is not None) and (C is not None):
                    A = find_matches(grammar, B, C, grammar_rules)
                    #print("A: "+str(A))
                    key = save_rules(A, None, parsing_matrix, i, j, key, grammar_rules)
    
    return parsing_matrix, grammar_rules


#Funzioni utili per la visualizzazione dei risultati

In [ ]:
def print_tree(idx, indentation):
    """
    Stampa l'abero/gli alberi di parsing
    """
    #se l'albero contiene una sola produzione quindi solo ad esempio Verb=book
    if not (grammar_rules[idx][2]):
        # rigo 1: S -> Verb=book
        print("\t" * indentation, grammar_rules[idx][0], "=", grammar_rules[idx][1])
    else:
        #casi con più produzioni
        print("\t" * indentation, grammar_rules[idx][0], "->")
        indentation += 1
        print_tree(int(grammar_rules[idx][1]), indentation)
        print_tree(int(grammar_rules[idx][2]), indentation)


def print_matrix():
    
    #Stampa la matrice del CKY

    #np.full restituisce un nuovo array della stessa grandezza dell'array che diamo come parametro nella chiamata
    final_matrix = np.full((len(words),len(words)+1), None)
    #scorro la matrice
    for idx1,row in enumerate(matrix):
        for idx2,cell in enumerate(row):
            value = []
            if cell:
                for key in cell:
                    value.append(grammar_rules[key][0])
            else:
                value = None
            final_matrix[idx1,idx2] = value
    #tabulate serve a stampare la matrice final matrix, a cui aggiungiamo contorni e indici
    print(tabulate(final_matrix[:,1:], headers=range(1,len(words)+1), showindex='always', tablefmt='fancy_grid'))




def print_result():
    """
    Stampa:
    - la frase data in input
    - la matrice generata
    - Gli alberi di parsing nel caso la frase appartenga alla grammatica
    """
    print("Data la seguente frase in input, vediamo il risultato prodotto:\n")
    print('\033[1m' + phrase + '\033[0m'+"\n")
    # stampa la matrice
    print_matrix()
    # dice se la frase appartiene o meno alla grammatica
    if (matrix[0,len(words)] is not None):
        num_trees = 0
        for id in matrix[0,len(words)]:
            #se arrivo alla S quindi alla radice allora sono riuscito a produrre un albero
            if grammar_rules[id][0] == "S":
                print("-------------------------------------------------------------------" * (num_trees > 0))
                #incremento il conteggio degli alberi prodotti
                num_trees += 1
                print_tree(id, 0)
        if (num_trees == 0):
            print("La frase %s appartiene alla grammatica." % ('\033[1m' + 'NON' + '\033[0m'))
    else:
        print("La frase %s appartiene alla grammatica." % ('\033[1m' + 'NON' + '\033[0m'))

#Parsificazione frasi in Inglese

In [ ]:
#r.sub sostituisce i caratteri nel primo parametro con quelli del secondo ''
phrase = re.sub(r'[^\w\s]', '', English_phrases[0])
words = phrase.split()
matrix, grammar_rules = CKY(words, English_Grammar)
print_result()

Data la seguente frase in input, vediamo il risultato prodotto:

Book the flight through Houston

╒════╤══════════════════════════════════╤═════════╤═════════════════════╤══════════╤══════════════════════════════════════════╕
│    │ 1                                │ 2       │ 3                   │ 4        │ 5                                        │
╞════╪══════════════════════════════════╪═════════╪═════════════════════╪══════════╪══════════════════════════════════════════╡
│  0 │ ['S', 'Nominal', 'Noun', 'Verb'] │         │ ['S', 'VP', 'VNP']  │          │ ['S', 'VP', 'VNP', 'S', 'VP', 'S', 'VP'] │
├────┼──────────────────────────────────┼─────────┼─────────────────────┼──────────┼──────────────────────────────────────────┤
│  1 │                                  │ ['Det'] │ ['NP']              │          │ ['NP']                                   │
├────┼──────────────────────────────────┼─────────┼─────────────────────┼──────────┼──────────────────────────────────────────┤
│  2 │

In [ ]:
phrase = re.sub(r'[^\w\s]', '', English_phrases[1])
words = phrase.split()
matrix, grammar_rules = CKY(words, English_Grammar)
print_result()

Data la seguente frase in input, vediamo il risultato prodotto:

Does she prefer a morning flight

╒════╤═════════╤═════════╤═══════════════╤═════════╤═════════════════════╤═════════════════════╕
│    │ 1       │ 2       │ 3             │ 4       │ 5                   │ 6                   │
╞════╪═════════╪═════════╪═══════════════╪═════════╪═════════════════════╪═════════════════════╡
│  0 │ ['Aux'] │ ['ANP'] │               │         │ ['S']               │ ['S']               │
├────┼─────────┼─────────┼───────────────┼─────────┼─────────────────────┼─────────────────────┤
│  1 │         │ ['NP']  │               │         │ ['S']               │ ['S']               │
├────┼─────────┼─────────┼───────────────┼─────────┼─────────────────────┼─────────────────────┤
│  2 │         │         │ ['S', 'Verb'] │         │ ['S', 'VP', 'VNP']  │ ['S', 'VP', 'VNP']  │
├────┼─────────┼─────────┼───────────────┼─────────┼─────────────────────┼─────────────────────┤
│  3 │         │         │  

#Parsificazione frasi in Dothraki

In [ ]:
phrase = re.sub(r'[^\w\s]', '', Dothraki_phrases[0])
words = phrase.split()
matrix, grammar_rules = CKY(words, Dothraki_Grammar)
print_result()

Data la seguente frase in input, vediamo il risultato prodotto:

Hash yer astoe ki Dothraki

╒════╤═════════╤═════════╤══════════╤══════════╤══════════╕
│    │ 1       │ 2       │ 3        │ 4        │ 5        │
╞════╪═════════╪═════════╪══════════╪══════════╪══════════╡
│  0 │ ['Aux'] │ ['ANP'] │ ['AV']   │          │ ['S']    │
├────┼─────────┼─────────┼──────────┼──────────┼──────────┤
│  1 │         │ ['NP2'] │          │          │          │
├────┼─────────┼─────────┼──────────┼──────────┼──────────┤
│  2 │         │         │ ['Verb'] │          │          │
├────┼─────────┼─────────┼──────────┼──────────┼──────────┤
│  3 │         │         │          │ ['Prep'] │ ['PN']   │
├────┼─────────┼─────────┼──────────┼──────────┼──────────┤
│  4 │         │         │          │          │ ['Noun'] │
╘════╧═════════╧═════════╧══════════╧══════════╧══════════╛

 S ->
	 AV ->
		 ANP ->
			 Aux = Hash
			 NP2 = yer
		 Verb = astoe
	 PN ->
		 Prep = ki
		 Noun = Dothraki


In [ ]:
phrase = re.sub(r'[^\w\s]', '', Dothraki_phrases[1])
words = phrase.split()
matrix, grammar_rules = CKY(words, Dothraki_Grammar)
print_result()

Data la seguente frase in input, vediamo il risultato prodotto:

Anha zhilak yera

╒════╤════════╤══════════╤════════╕
│    │ 1      │ 2        │ 3      │
╞════╪════════╪══════════╪════════╡
│  0 │ ['NP'] │ ['S']    │ ['S']  │
├────┼────────┼──────────┼────────┤
│  1 │        │ ['Verb'] │ ['VP'] │
├────┼────────┼──────────┼────────┤
│  2 │        │          │ ['NP'] │
╘════╧════════╧══════════╧════════╛

 S ->
	 NP = Anha
	 VP ->
		 Verb = zhilak
		 NP = yera


In [ ]:
phrase = re.sub(r'[^\w\s]', '', Dothraki_phrases[2])
words = phrase.split()
matrix, grammar_rules = CKY(words, Dothraki_Grammar)
print_result()

Data la seguente frase in input, vediamo il risultato prodotto:

Anha gavork

╒════╤════════╤══════════╕
│    │ 1      │ 2        │
╞════╪════════╪══════════╡
│  0 │ ['NP'] │ ['S']    │
├────┼────────┼──────────┤
│  1 │        │ ['Noun'] │
╘════╧════════╧══════════╛

 S ->
	 NP = Anha
	 Noun = gavork
